<a href="https://colab.research.google.com/github/renatavel/goit_ds_homeworks/blob/main/hw_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [4]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [5]:
X_train = X_train.astype('float32') / 255.0
X_val = X_val.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

In [6]:
X_train = X_train.reshape(-1, 28, 28)
X_val = X_val.reshape(-1, 28, 28)
X_test = X_test.reshape(-1, 28, 28)

In [7]:
y_train_hot = keras.utils.to_categorical(y_train, 10)
y_val_hot = keras.utils.to_categorical(y_val, 10)
y_test_hot = keras.utils.to_categorical(y_test, 10)

In [8]:
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, LeakyReLU, Input, BatchNormalization
from keras.regularizers import l2

model = Sequential()
model.add(Input(shape=(28, 28)))
model.add(Flatten())

model.add(Dense(512, kernel_regularizer=l2(0.0001)))
model.add(LeakyReLU(negative_slope=0.1))

model.add(Dense(256, kernel_regularizer=l2(0.0001)))
model.add(LeakyReLU(negative_slope=0.1))

model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [9]:
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0005) , loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train_hot, epochs=40, batch_size=64, validation_data=(X_val, y_val_hot), verbose=1)

Epoch 1/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.7285 - loss: 0.8683 - val_accuracy: 0.8533 - val_loss: 0.5040
Epoch 2/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8526 - loss: 0.5010 - val_accuracy: 0.8628 - val_loss: 0.4619
Epoch 3/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.8669 - loss: 0.4567 - val_accuracy: 0.8774 - val_loss: 0.4323
Epoch 4/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.8745 - loss: 0.4244 - val_accuracy: 0.8772 - val_loss: 0.4229
Epoch 5/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8805 - loss: 0.4031 - val_accuracy: 0.8716 - val_loss: 0.4239
Epoch 6/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.8837 - loss: 0.3909 - val_accuracy: 0.8830 - val_loss: 0.4002
Epoch 7/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8906 - loss: 0.3712 - val_accuracy: 0.8774 - val_loss: 0.4085
Epoch 8/40
750/750 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8946 - loss: 0.3578 - val_accuracy: 0.

In [11]:
test_loss, test_accuracy = model.evaluate(X_test, y_test_hot)
print(f"Test loss: {test_loss}")
print(f"Test accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 2s 5ms/step - accuracy: 0.8818 - loss: 0.4216
Test loss: 0.4190298914909363
Test accuracy: 0.8827999830245972


In [13]:
def build_model(hp):
    model = Sequential()
    model.add(Input(shape=(28, 28)))
    model.add(Flatten())

    model.add(Dense(
        units=hp.Int("units_layer1", min_value=256, max_value=512, step=128),
        kernel_regularizer=l2(hp.Choice("l2_reg_layer1", [0.0001, 0.001, 0.01]))
    ))
    model.add(LeakyReLU(negative_slope=0.1))

    model.add(Dense(
        units=hp.Int("units_layer2", min_value=128, max_value=256, step=64),
        kernel_regularizer=l2(hp.Choice("l2_reg_layer2", [0.0001, 0.001, 0.01]))
    ))
    model.add(LeakyReLU(negative_slope=0.1))

    model.add(Dropout(hp.Float("dropout_rate", min_value=0.2, max_value=0.5, step=0.05)))

    model.add(Dense(10, activation="softmax"))
    model.compile(
        optimizer=keras.optimizers.Adam(
            learning_rate=hp.Choice("learning_rate", [0.001, 0.0005, 0.0001])
        ),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    return model

In [14]:
import keras_tuner as kt
tuner = kt.Hyperband(
    build_model,
    objective="val_accuracy",
    max_epochs=40,
    factor=3,
    directory="tuner_dir",
    project_name="keras_tuning"
)
tuner.search(X_train, y_train, epochs=10, validation_data=(X_test, y_test), verbose=1)
best_parameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print(best_parameters)


Trial 90 Complete [00h 03m 08s]
val_accuracy: 0.8912000060081482

Best val_accuracy So Far: 0.8912000060081482
Total elapsed time: 01h 07m 31s
